# Week 2 Exercise - Simple Technical Tutor

A simple prototype with Gradio UI, streaming, model switch (Llama/OpenRouter), SQLite tool calling, and optional image input.

In [30]:
# imports
import os
import json
import base64
import mimetypes
import sqlite3
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI

In [31]:
# setup
MODEL_LLAMA = "llama3.2"
MODEL_OPENROUTER = "openai/gpt-4o-mini"
DB_PATH = "week2_kb.db"

load_dotenv(override=True)
ollama_client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
openrouter_client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=os.getenv("OPENROUTER_API_KEY"))

In [32]:
# database and tools
def init_db():
    with sqlite3.connect(DB_PATH) as conn:
        cur = conn.cursor()
        cur.execute("CREATE TABLE IF NOT EXISTS knowledge (topic TEXT PRIMARY KEY, answer TEXT NOT NULL)")
        seed = [
            ("rag", "RAG retrieves relevant external context before generation to improve grounding."),
            ("embedding", "Embeddings convert text into vectors to support semantic similarity search."),
            ("hallucination", "Hallucination is when the model generates unsupported or incorrect information."),
            ("vector database", "A vector database stores embeddings and retrieves nearest neighbors for RAG.")
        ]
        cur.executemany("INSERT OR IGNORE INTO knowledge(topic, answer) VALUES (?, ?)", seed)
        conn.commit()

def lookup_topic(topic):
    with sqlite3.connect(DB_PATH) as conn:
        cur = conn.cursor()
        cur.execute("SELECT topic, answer FROM knowledge WHERE lower(topic)=lower(?)", (topic,))
        row = cur.fetchone()
    if row:
        return {"topic": row[0], "answer": row[1]}
    return {"topic": topic, "answer": "No database entry found."}

def save_topic(topic, answer):
    with sqlite3.connect(DB_PATH) as conn:
        cur = conn.cursor()
        cur.execute("INSERT INTO knowledge(topic, answer) VALUES (?, ?) ON CONFLICT(topic) DO UPDATE SET answer=excluded.answer", (topic, answer))
        conn.commit()
    return {"status": "saved", "topic": topic}

init_db()

tools = [
    {
        "type": "function",
        "function": {
            "name": "lookup_topic",
            "description": "Look up a technical AI topic in the SQLite knowledge base.",
            "parameters": {
                "type": "object",
                "properties": {"topic": {"type": "string"}},
                "required": ["topic"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "save_topic",
            "description": "Save or update a technical topic in the SQLite knowledge base.",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {"type": "string"},
                    "answer": {"type": "string"}
                },
                "required": ["topic", "answer"],
                "additionalProperties": False
            }
        }
    }
]

TOOL_REGISTRY = {"lookup_topic": lookup_topic, "save_topic": save_topic}

def execute_tool_call(tool_call, tool_activity):
    name = tool_call.function.name
    try:
        args = json.loads(tool_call.function.arguments or "{}")
        result = TOOL_REGISTRY[name](**args)
        tool_activity.append(f"{name}({args}) -> {result}")
        return {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
    except Exception as e:
        err = {"error": str(e), "tool": name}
        tool_activity.append(f"{name} error -> {err}")
        return {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(err)}

In [33]:
# prompts and streaming
system_prompt = "You are a technical tutor for AI and RAG. Use concise explanations with sections: Short answer, How it works, Example, Analogy. Use tool results when available."

def image_to_data_url(path):
    if not path:
        return None
    mime = mimetypes.guess_type(path)[0] or "image/png"
    with open(path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode("utf-8")
    return f"data:{mime};base64,{b64}"

def build_messages(question, image_path=None):
    if image_path:
        data_url = image_to_data_url(image_path)
        user_content = [
            {"type": "text", "text": question},
            {"type": "image_url", "image_url": {"url": data_url}}
        ]
    else:
        user_content = question
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_content}
    ]

def stream_llama(question, image_path=None):
    text_question = question if not image_path else question + " (User uploaded an image, but local Llama path is text-only.)"
    messages = build_messages(text_question, image_path=None)
    stream = ollama_client.chat.completions.create(model=MODEL_LLAMA, messages=messages, temperature=0.2, stream=True)
    result = ""
    tool_trace = "No tools used (Llama path)."
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result, tool_trace

def stream_openrouter(question, image_path=None):
    if not os.getenv("OPENROUTER_API_KEY"):
        yield "OPENROUTER_API_KEY is missing. Add it to your .env file.", ""
        return

    messages = build_messages(question, image_path=image_path)
    tool_activity = []
    response = openrouter_client.chat.completions.create(model=MODEL_OPENROUTER, messages=messages, tools=tools, temperature=0.2)

    iterations = 0
    while response.choices[0].finish_reason == "tool_calls" and iterations < 5:
        iterations += 1
        message = response.choices[0].message
        tool_messages = [execute_tool_call(tc, tool_activity) for tc in (message.tool_calls or [])]
        messages.append(message)
        messages.extend(tool_messages)
        response = openrouter_client.chat.completions.create(model=MODEL_OPENROUTER, messages=messages, tools=tools, temperature=0.2)

    tool_trace = "\n".join(tool_activity) if tool_activity else "No tools were called."
    stream = openrouter_client.chat.completions.create(model=MODEL_OPENROUTER, messages=messages, temperature=0.2, stream=True)
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result, tool_trace

def stream_answer(question, model_choice, image_path):
    if not question.strip():
        yield "Please enter a technical question.", ""
        return
    if model_choice == "Llama":
        yield from stream_llama(question, image_path=image_path)
    elif model_choice == "OpenRouter":
        yield from stream_openrouter(question, image_path=image_path)
    else:
        yield "Unknown model selected.", ""

In [ ]:
# gradio ui
question_input = gr.Textbox(label="Your technical question", lines=4)
model_selector = gr.Dropdown(["Llama", "OpenRouter"], label="Select model", value="OpenRouter")
image_input = gr.Image(type="filepath", label="Optional image (multimodal)")
answer_output = gr.Markdown(label="Answer")
tool_output = gr.Textbox(label="Tool Activity", lines=8)

view = gr.Interface(
    fn=stream_answer,
    inputs=[question_input, model_selector, image_input],
    outputs=[answer_output, tool_output],
    title="Week 2 Technical Tutor",
    description="Simple streaming tutor with model switch, SQLite tools, optional image input, and tool trace panel.",
    examples=[
        ["What is RAG and why does it reduce hallucinations?", "OpenRouter", None],
        ["Look up embedding in the knowledge base and explain it with an analogy.", "OpenRouter", None],
        ["What is a vector database?", "Llama", None]
    ],
    flagging_mode="never"
)

view.launch()